# Movielens SVD collaborative filtering example
This example will use the 1M Movielens dataset (http://grouplens.org/datasets/movielens/1m/).

This dataset contains ~1,000,000 ratings from ~6000 users on ~3900 movies.

## Download and extract dataset
We will use the raw text dataset. We fill first download the ZIP if not done already and then extract it

In [1]:
from urllib.request import urlretrieve
from zipfile import ZipFile
import os

srcUrl = 'http://files.grouplens.org/datasets/movielens/ml-1m.zip'

# Create folder if it doesn't exists
if not os.path.exists('./movielens-1m'):
    print("Create movielens-1m folder")
    os.makedirs('./movielens-1m')

# Check for existance of ZIP file
if not os.path.exists('./movielens-1m/ml-1m.zip'):
    print("Download %s" % srcUrl)
    urlretrieve(srcUrl, './movielens-1m/ml-1m.zip')
    
# Extract zipFile
with ZipFile('./movielens-1m/ml-1m.zip', 'r') as zipFile:
    print("Extract %d files from ml-1m.zip" % len(zipFile.namelist()))
    zipFile.extractall('./movielens-1m')

Extract 5 files from ml-1m.zip


## Create dataframes
Extract the data from the individual files and created pandas DataFrame's from them

In [ ]:
import pandas as pd
import numpy as np

ratings_list = [i.strip().split("::") for i in open('./movielens-1m/ml-1m/ratings.dat', 'r', encoding='iso-8859-1').readlines()]
users_list = [i.strip().split("::") for i in open('./movielens-1m/ml-1m/users.dat', 'r', encoding='iso-8859-1').readlines()]
movies_list = [i.strip().split("::") for i in open('./movielens-1m/ml-1m/movies.dat', 'r', encoding='iso-8859-1').readlines()]

ratings_df = pd.DataFrame(ratings_list, columns = ['UserID', 'MovieID', 'Rating', 'Timestamp'], dtype = int)
movies_df = pd.DataFrame(movies_list, columns = ['MovieID', 'Title', 'Genres'])
movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [43]:
movies_df.head()

,MovieID,Title,Genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
ratings_df.head()

,UserID,MovieID,Rating,Timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


## Split into a training and test set
Split the data into a training and a test set. We use a split of 25% in this case

In [45]:
# Initialze a RandomState with a constant seed to make the split consistent
from numpy.random import RandomState
prng = RandomState(1)

# Split the dataset into a training and test set
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ratings_df, test_size = .25, random_state=prng)

## Create pivot tables
Create pivot tables for training and testset. On one axis are the UserID's on the other axis are the MovieIds. Fill in the empty values with zero's

In [ ]:
R_train_df = train_data.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_test_df = test_data.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)

R_train_df.head()

## De-mean the data
Every user rates things differently, where one gives a 5 star rating if he just liked the movie some other users may never give a 4 or higher so average out the data.

In [ ]:
R_train_df.as_matrix().shape
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

## Perform the Singular Value Decomposition
Use scipy to do the Singular Value Decomposition. Limit the number of latent factors to 50.

In [59]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 100)

sigma = np.diag(sigma)

## Use the Decomposed Matrices to make the predicted ratings
With U, sigma and Vt we can recreate the matrix using k (50) latent factors. Then re-add the user mean

In [60]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,3.637457,0.162269,0.087712,0.087208,0.110059,-0.144687,0.049394,0.108098,0.013329,0.061780,...,-0.031152,0.028462,0.018501,-0.033629,-0.030366,0.393260,-0.103090,0.044514,0.030920,0.053901
1,0.020086,0.228464,0.075369,-0.045735,0.009402,0.576817,-0.136306,0.059655,0.012000,1.003151,...,0.012940,-0.019715,-0.019530,0.063508,-0.027315,0.007177,-0.080022,0.026943,-0.058286,0.112306
2,1.087179,0.275231,-0.073749,0.016847,0.018586,0.020935,-0.048331,0.041952,0.022030,0.682441,...,-0.026490,-0.010436,0.020250,0.050102,-0.056764,0.113136,0.069312,0.072299,-0.006637,0.120152
3,-0.434535,0.015223,-0.034736,0.066534,0.018288,0.274640,-0.107499,0.013341,0.001894,-0.307092,...,-0.037843,-0.011654,-0.017948,-0.056821,-0.060781,0.057800,-0.060168,-0.077854,0.000365,-0.071464
4,0.344137,-0.332776,0.027825,0.253913,-0.020563,0.909716,-0.030355,0.027459,-0.141792,0.197430,...,0.081974,0.051466,0.036577,-0.083680,-0.023133,0.022863,0.348144,-0.020243,0.097504,-0.116012


## Compare results against our test set

In [61]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print('Predictions RMSE: %0.2f' % rmse(all_user_predicted_ratings, R_test_df.as_matrix()))

Predictions RMSE: 3.54


## Get recommended movie list

In [26]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print('User %d has already rated %d movies.' % (userID, user_full.shape[0]))
    print('Recommending the highest %d predicted ratings movies not already rated.' % num_recommendations)
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations, sorted_user_predictions

# Get recommendations
already_rated, recommendations, sorted_user_predictions = recommend_movies(preds_df, 5, movies_df, ratings_df, 10)

In [35]:
already_rated.head(10)

,UserID,MovieID,Rating,Timestamp,Title,Genres
72,5,2571,5,978244493,"Matrix, The (1999)",Action|Sci-Fi|Thriller
172,5,29,5,978245065,"City of Lost Children, The (1995)",Adventure|Sci-Fi
116,5,2599,5,978242323,Election (1999),Comedy
114,5,1213,5,978244177,GoodFellas (1990),Crime|Drama
142,5,1089,5,978244205,Reservoir Dogs (1992),Crime|Thriller
28,5,2997,5,978241556,Being John Malkovich (1999),Comedy
144,5,913,5,978242740,"Maltese Falcon, The (1941)",Film-Noir|Mystery
46,5,1046,5,978244114,Beautiful Thing (1996),Drama|Romance
150,5,1732,5,978245740,"Big Lebowski, The (1998)",Comedy|Crime|Mystery|Thriller
21,5,1250,5,978241112,"Bridge on the River Kwai, The (1957)",Drama|War


In [36]:
recommendations.head(10)

,MovieID,Title,Genres
2137,2336,Elizabeth (1998),Drama
202,223,Clerks (1994),Comedy
2928,3175,Galaxy Quest (1999),Adventure|Comedy|Sci-Fi
1535,1673,Boogie Nights (1997),Drama
2868,3114,Toy Story 2 (1999),Animation|Children's|Comedy
721,778,Trainspotting (1996),Drama
212,235,Ed Wood (1994),Comedy|Drama
420,457,"Fugitive, The (1993)",Action|Thriller
1019,1094,"Crying Game, The (1992)",Drama|Romance|War
2190,2396,Shakespeare in Love (1998),Comedy|Romance
